In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/twitter-entity-sentiment-analysis/twitter_validation.csv
/kaggle/input/twitter-entity-sentiment-analysis/twitter_training.csv


In [2]:
import pandas as pd
import numpy as np


In [3]:
columns = ['id','country','Label','Text']
df=pd.read_csv('/kaggle/input/twitter-entity-sentiment-analysis/twitter_training.csv',names=columns)

In [4]:
df.head()

,id,country,Label,Text
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [5]:
df.shape

(74682, 4)

In [6]:
df.dtypes

id          int64
country    object
Label      object
Text       object
dtype: object

In [7]:
list_text=df['Text'].tolist()

In [8]:
len(list_text)

74682

In [9]:
from transformers import pipeline
from transformers import AutoTokenizer,AutoModelForSequenceClassification

In [10]:
model_name='distilbert-base-uncased-finetuned-sst-2-english'

In [11]:
model=AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer=AutoTokenizer.from_pretrained(model_name)

In [12]:
classifier=pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

In [13]:
#ressult_score=[]
#for i in range(len(df['Text'])):
results=classifier(list(df['Text'][:35]))
results

[{'label': 'NEGATIVE', 'score': 0.9834704995155334},
 {'label': 'POSITIVE', 'score': 0.9932871460914612},
 {'label': 'POSITIVE', 'score': 0.9783738851547241},
 {'label': 'NEGATIVE', 'score': 0.9768757820129395},
 {'label': 'POSITIVE', 'score': 0.701887845993042},
 {'label': 'NEGATIVE', 'score': 0.9805619120597839},
 {'label': 'NEGATIVE', 'score': 0.7845344543457031},
 {'label': 'NEGATIVE', 'score': 0.9357830286026001},
 {'label': 'POSITIVE', 'score': 0.9987744688987732},
 {'label': 'NEGATIVE', 'score': 0.9372895956039429},
 {'label': 'NEGATIVE', 'score': 0.9648422598838806},
 {'label': 'POSITIVE', 'score': 0.7795964479446411},
 {'label': 'POSITIVE', 'score': 0.9972885847091675},
 {'label': 'POSITIVE', 'score': 0.9972885847091675},
 {'label': 'POSITIVE', 'score': 0.9980003237724304},
 {'label': 'POSITIVE', 'score': 0.998803973197937},
 {'label': 'POSITIVE', 'score': 0.9034885764122009},
 {'label': 'NEGATIVE', 'score': 0.9396255612373352},
 {'label': 'POSITIVE', 'score': 0.99944120645523

In [14]:
token=tokenizer.tokenize(df['Text'][0])
token

['im',
 'getting',
 'on',
 'border',
 '##lands',
 'and',
 'i',
 'will',
 'murder',
 'you',
 'all',
 ',']

In [15]:
token_id=tokenizer.convert_tokens_to_ids(token)
token_id

[10047, 2893, 2006, 3675, 8653, 1998, 1045, 2097, 4028, 2017, 2035, 1010]

In [16]:
input_ids=tokenizer(df['Text'][0])
input_ids

{'input_ids': [101, 10047, 2893, 2006, 3675, 8653, 1998, 1045, 2097, 4028, 2017, 2035, 1010, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [17]:
X_train=list(df['Text'][:10])

In [18]:
batch=tokenizer(X_train,padding=True,truncation=True,max_length=512,return_tensors='pt')
batch

{'input_ids': tensor([[  101, 10047,  2893,  2006,  3675,  8653,  1998,  1045,  2097,  4028,
          2017,  2035,  1010,   102,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0],
        [  101,  1045,  2572,  2746,  2000,  1996,  6645,  1998,  1045,  2097,
          3102,  2017,  2035,  1010,   102,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,


In [19]:
import torch
import torch.nn.functional as F

In [20]:
with torch.no_grad():
    outputs=model(**batch)
    print(outputs)
    predictions=F.softmax(outputs.logits, dim=1)
    print(predictions)
    labels=torch.argmax(predictions,dim=1)
    print(labels)
    labels=[model.config.id2label[label_id] for label_id in labels.tolist()]
    print(labels)

SequenceClassifierOutput(loss=None, logits=tensor([[ 2.2075, -1.8785],
        [-2.4459,  2.5511],
        [-1.8743,  1.9377],
        [ 2.0326, -1.7109],
        [-0.3719,  0.4844],
        [ 2.1153, -1.8056],
        [ 0.6950, -0.5973],
        [ 1.4526, -1.2266],
        [-3.2673,  3.4359],
        [ 1.4471, -1.2574]]), hidden_states=None, attentions=None)
tensor([[0.9835, 0.0165],
        [0.0067, 0.9933],
        [0.0216, 0.9784],
        [0.9769, 0.0231],
        [0.2981, 0.7019],
        [0.9806, 0.0194],
        [0.7845, 0.2155],
        [0.9358, 0.0642],
        [0.0012, 0.9988],
        [0.9373, 0.0627]])
tensor([0, 1, 1, 0, 1, 0, 0, 0, 1, 0])
['NEGATIVE', 'POSITIVE', 'POSITIVE', 'NEGATIVE', 'POSITIVE', 'NEGATIVE', 'NEGATIVE', 'NEGATIVE', 'POSITIVE', 'NEGATIVE']


In [21]:
df['Label'][:10].tolist()

['Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive']

In [22]:
df.head()

,id,country,Label,Text
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [23]:
import re

In [24]:

def clean_text(text):
    # Remove special characters and punctuation
    text = re.sub(r'[^A-Za-z0-9\s]', '', str(text))
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text



In [25]:
df['Preprocessed Text'] = df['Text'].apply(clean_text) 


In [26]:
df.head()

,id,country,Label,Text,Preprocessed Text
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...,i am coming to the borders and i will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...,im getting on borderlands and i will kill you all
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...,im getting on borderlands 2 and i will murder ...


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74682 entries, 0 to 74681
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 74682 non-null  int64 
 1   country            74682 non-null  object
 2   Label              74682 non-null  object
 3   Text               73996 non-null  object
 4   Preprocessed Text  74682 non-null  object
dtypes: int64(1), object(4)
memory usage: 2.8+ MB


In [28]:
X_train=list(df['Preprocessed Text'][:1000])

In [29]:
batch=tokenizer(X_train,padding=True,truncation=True,max_length=512,return_tensors='pt')
#batch

In [30]:
with torch.no_grad():
    outputs=model(**batch)
    #print(outputs)
    predictions=F.softmax(outputs.logits, dim=1)
    #print(predictions)
    labels=torch.argmax(predictions,dim=1)
    #print(labels)
    labels=[model.config.id2label[label_id] for label_id in labels.tolist()]
    #print(labels)

In [31]:
len(labels)

1000

In [32]:
new_df=df[:1000]

In [33]:
new_df['Label1']=labels

/tmp/ipykernel_21/728350726.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Label1']=labels


In [34]:
new_df.head()

,id,country,Label,Text,Preprocessed Text,Label1
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...,im getting on borderlands and i will murder yo...,NEGATIVE
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...,i am coming to the borders and i will kill you...,POSITIVE
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...,im getting on borderlands and i will kill you all,POSITIVE
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...,im coming on borderlands and i will murder you...,NEGATIVE
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...,im getting on borderlands 2 and i will murder ...,NEGATIVE


In [35]:
new_df['Label'].value_counts()

Label
Positive      424
Neutral       282
Negative      192
Irrelevant    102
Name: count, dtype: int64

In [36]:
new_df['Label1'].value_counts()

Label1
NEGATIVE    526
POSITIVE    474
Name: count, dtype: int64